################################################################################################################################
#           Assignment 2: Author-Vaishali Lambe, NUID-001286444                            #
################################################################################################################################
** Question 2:**

**Part 2**
* Read all the json files in the folder called Data.
* Read only the json files which contain the key called “restaurants”
* Each (or most of the json files) contain a key called “open” which contains the details of the operation (timings) of the restaurants. For each json file, read the timings of the restaurants.
* Data of the operation timings of the restaurants is present for each day of the week. I want you to extract each of this data and write it in an excel sheet.

**Bonus:**
* Split “Start Time” column into two different columns having the hour and minute in each of them
* Split “End Time” column into two different columns having the hour and minute in each of them

################################################################################################################################

In [1]:
import os, json

# Assume that the JSON files live in a 'Data' directory at the same level as this Jupyter notebook
path_to_json = os.path.join('.', 'Data', '')
filenames = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(filenames[0:20])

['1-abercrombie-lane-sydney.json', '1-chicken-rice-and-seafood-houston.json', '1-chinese-kitchen-san-jose-2.json', '10-custom-pizzeria-braddon.json', '10-downing-street-london.json', '10-greek-street-london.json', '100-club-london.json', '100-de-agave-denver.json', '100-hoxton-london.json', '109-adelaide.json', '11-inch-pizza-melbourne.json', '111-by-nico-glasgow-2.json', '127-bar-and-bistro-brisbane.json', '13-north-grille-phoenix.json', '13th-note-glasgow.json', '140-perth-perth.json', '1428-haight-patio-cafe-and-crepery-san-francisco.json', '143-photo-booths-san-jose.json', '163-vietnamese-sandwiches-and-bubble-tea-boston.json', '1760-san-francisco.json']


In [2]:
import shutil

restaurant_files = []
for f in filenames:
    file_path = os.path.join(path_to_json, f)
    with open(file_path) as json_file:
        # Load the content of the file.        
        json_content = json.load(json_file)
        
        country = json_content['location']['country']
        #print(country)
        
        city = json_content['location']['city']
        #print(city)
        
        term = json_content['term']
        #print(term)
        
        if term == 'restaurants':
            restaurant_files.append(file_path)

#print(restaurant_files)

In [3]:
import pandas as pd

# Function for processing the JSON content of a single file
# json_content is the file content, created using json.load(file)
# row_list is the list of rows to which a row will be added; later used to create a dataframe
# exception_list is the list of files that couldn't be processed
def process_restaurant_file_json(json_content, row_list, exception_list):
    name = json_content['name']
    city = json_content['location']['city']
    country = json_content['location']['country']        
    #print(name + ', ' + city + ', ' + country)
        
    # Extract the opening hours, if there are any.
    if 'hours' in json_content:
        for hours in json_content['hours']:
            if hours['hours_type'] == 'REGULAR':
                for h in hours['open']:
                    day = str(h['day'])
                    start = str(h['start'])
                    end = str(h['end'])
                    #print('Day ' + day + ': ' + start + ' - ' + end)
                    
                    # Split the hours and minutes.
                    start_hour = start[0:2]
                    start_minutes = start[2:4]

                    end_hour = end[0:2]
                    end_minutes = end[2:4]

                    if (len(start_hour) != 2) or (len(start_minutes) != 2) or (len(end_hour) != 2) or (len(end_minutes) != 2):
                        print(start_hour + ' ' + start_minutes + ' ' + end_hour + ' ' + end_minutes)
                    row = {'Name of restaurant':name, 'City':city, 'Country Code':country, 
                           'Day of week':day, 'Start time hour':start_hour, 'Start time minute':start_minutes, 
                           'End time hour':end_hour, 'End time minute':end_minutes}
                    row_list.append(row)
            else:
                print(name + ' has hours_type of ' + hours['hours_type'])
    else:
        # Save the exceptions for separate examination.
        # Maybe they just have 'hours_type' that isn't REGULAR? No, they don't.
        # The files in the exception list have no 'hours' at all, so leave them out.
        #print('no opening hours found for: ' + rf)
        exception_list.append(rf)
        
    return row_list, exception_list

In [4]:
exceptions = []
row_list = []

for rf in restaurant_files:
    with open(rf) as json_file:
        # Load the content of the file.        
        json_content = json.load(json_file)
        
        # Process the JSON content.
        row_list, exceptions = process_restaurant_file_json(json_content, row_list, exceptions)

# Create the dataframe from the list of rows.
df = pd.DataFrame(row_list, columns=['Name of restaurant', 'City', 'Country Code', 'Day of week', 
                           'Start time hour', 'Start time minute', 'End time hour', 'End time minute'])

# Examine the dataframe.
print(df.head(20))

# Write the CSV file.  Don't write the row numbers (index).
df.to_csv(os.path.join('.', 'restaurants.csv'), index=False)

           Name of restaurant      City Country Code Day of week  \
0          1 Abercrombie Lane    Sydney           AU           0   
1          1 Abercrombie Lane    Sydney           AU           1   
2          1 Abercrombie Lane    Sydney           AU           2   
3          1 Abercrombie Lane    Sydney           AU           3   
4          1 Abercrombie Lane    Sydney           AU           4   
5   #1 Chicken Rice & Seafood   Houston           US           0   
6   #1 Chicken Rice & Seafood   Houston           US           1   
7   #1 Chicken Rice & Seafood   Houston           US           2   
8   #1 Chicken Rice & Seafood   Houston           US           3   
9   #1 Chicken Rice & Seafood   Houston           US           4   
10  #1 Chicken Rice & Seafood   Houston           US           5   
11         #1 Chinese Kitchen  San Jose           US           0   
12         #1 Chinese Kitchen  San Jose           US           2   
13         #1 Chinese Kitchen  San Jose         

In [5]:
# Examine the exceptions.
# All files: 11728
# Restaurant files: 6372
# Exceptions: 720
print('All files: ' + str(len(filenames)))
print('Restaurant files: ' + str(len(restaurant_files)))
print('Exceptions: ' + str(len(exceptions)))
print(exceptions[0:20])

for e in exceptions:
    with open(e) as json_file:
        json_content = json.load(json_file)
        
        name = None
        if 'name' in json_content:
            name = json_content['name']
        city = json_content['location']['city']
        country = json_content['location']['country']        
        #print(name + ', ' + city + ', ' + country)
        
        # Extract the opening hours.
        if 'hours' in json_content:
            print(name + ' has hours')
        else:
            print(name + ' has no hours')

All files: 11728
Restaurant files: 6372
Exceptions: 720
['.\\Data\\140-perth-perth.json', '.\\Data\\2-yummy-belconnen.json', '.\\Data\\3-little-monkeys-melbourne.json', '.\\Data\\369-oriental-bistro-houston.json', '.\\Data\\39-steps-o-connor.json', '.\\Data\\7-greens-detroit-salad-detroit.json', '.\\Data\\a-and-l-ham-palace-detroit.json', '.\\Data\\a-bite-to-eat-a-drink-as-well-chifley.json', '.\\Data\\abells-kopi-tiam-manuka.json', '.\\Data\\adams-cafe-liverpool.json', '.\\Data\\african-kitchen-gallery-london.json', '.\\Data\\ainslie-bar-and-bistro-mercure-braddon.json', '.\\Data\\alaras-turkish-pide-kebab-house-dickson.json', '.\\Data\\ali-baba-lebanese-restaurant-canberra.json', '.\\Data\\almond-restaurant-and-bar-liverpool.json', '.\\Data\\alto-acton.json', '.\\Data\\andells-restaraunt-and-tea-room-glasgow.json', '.\\Data\\andrew-square-house-of-pizza-boston.json', '.\\Data\\antojitos-salvadorenos-houston-2.json', '.\\Data\\aquila-caffe-bar-brisbane.json']
140 Perth has no hours
2 